# Optibook Manual

This notebook contains examples for **all interactions** you can do with optibook (that means inserting orders, getting your positions, etc.). You should use this notebook as a reference/documenation for the system later on when you write more extensive algorithms.

First we do some setup and import the optibook client, which is used to connect to the optibook exchange. If everything is setup correctly, you should see the line "Setup was successful." being printed.

In [1]:
from optibook.synchronous_client import Exchange

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


### Define Your Instrument

An instrument is the term we use for the 'thing' that we are trading. As such, if we are trading BMW stocks, we would call those an instrument. However, BMW options would be a different instrument. 

This field determines which instrument we are trading. By changing it, you will insert trades for a different instrument. You can see all available instruments by looking at the dropdown menu labeled "Instruments" on the visualizer.

In [15]:
# Ibro: ovo sam ja promjenio cisto da vidimo outpute svih cellova
instrument_id = "C1_FOSSIL_FUEL_ETF" # 'QUESTION_1'

### Connect to Exchange

In [16]:
e = Exchange()
a = e.connect()

# you can also define host/user/pass yourself
# when not defined, it is taken from ~/.optibook file if it exists
# if that file does not exists, an error is thrown

#e = Exchange(host='host-to-connect-to')
#a = e.connect(username='your-username', password='your-password')


2021-09-25 10:49:10,315 [asyncio   ] [MainThread  ] Using selector: EpollSelector


## Outstanding Orders, Trades, Current Positions and PnL

In [43]:
# Returns all currently outstanding orders
orders = e.get_outstanding_orders(instrument_id)
for o in orders.values():
    print(o)

AssertionError: Cannot call function until connected. Call connect() first

In [18]:
# Returns all trades you have done since the last time this function was called
trades = e.poll_new_trades(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [19]:
# Returns all trades you have done since since the instantiation of the Exchange
trades = e.get_trade_history(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [20]:
# Returns all current positions
positions = e.get_positions()
for p in positions:
    print(p, positions[p])

C1_GAS_INC 0
C1_OIL_CORP 0
C1_FOSSIL_FUEL_ETF 0
C2_GREEN_ENERGY_ETF 0
C2_SOLAR_CO 0
C2_WIND_LTD 0


In [21]:
# Returns all current positions with cash invested
positions = e.get_positions_and_cash()
for p in positions:
    print(p, positions[p])

C1_GAS_INC {'volume': 0, 'cash': 0.0}
C1_OIL_CORP {'volume': 0, 'cash': 0.0}
C1_FOSSIL_FUEL_ETF {'volume': 0, 'cash': 0.0}
C2_GREEN_ENERGY_ETF {'volume': 0, 'cash': -7.1000000000000085}
C2_SOLAR_CO {'volume': 0, 'cash': 0.0}
C2_WIND_LTD {'volume': 0, 'cash': 0.0}


In [22]:
# Returns Current PnL based on last Traded Price
pnl = e.get_pnl()
print(pnl)

-7.1000000000000085


## Order Book and Public Trade Ticks

In [23]:
book = e.get_last_price_book(instrument_id)
print(book)

In [41]:
# Ibro: dodao ovo jer ovo gore samo printa object
# Ovo su svi argumenti ovog objekta
print("Bids:\n", book.bids, "\n")
print("Asks:\n", book.asks, "\n")
print("Instrument ID:\n", book.instrument_id, "\n")
print("Timestamp:\n", book.timestamp, "\n")

Bids:
 [[price_volume] price=166.8, volume=250, [price_volume] price=166.70000000000002, volume=1500] 

Asks:
 [[price_volume] price=167.60000000000002, volume=250, [price_volume] price=168.10000000000002, volume=1500] 

Instrument ID:
 C1_FOSSIL_FUEL_ETF 

Timestamp:
 2021-09-25 10:49:12.733854 



In [42]:
# Returns all public tradeticks since the last time this function was called
tradeticks = e.poll_new_trade_ticks(instrument_id)
for t in tradeticks:
    print(f"[{t.instrument_id}] price({t.price}), volume({t.volume}), aggressor_side({t.aggressor_side}), buyer({t.buyer}), seller({t.seller})")

[C1_FOSSIL_FUEL_ETF] price(3000.0), volume(21), aggressor_side(ask), buyer(), seller()
[C1_FOSSIL_FUEL_ETF] price(166.8), volume(10), aggressor_side(ask), buyer(), seller()
[C1_FOSSIL_FUEL_ETF] price(166.8), volume(10), aggressor_side(ask), buyer(), seller()
[C1_FOSSIL_FUEL_ETF] price(167.5), volume(1), aggressor_side(bid), buyer(), seller()
[C1_FOSSIL_FUEL_ETF] price(167.5), volume(15), aggressor_side(bid), buyer(), seller()
[C1_FOSSIL_FUEL_ETF] price(166.9), volume(22), aggressor_side(ask), buyer(), seller()
[C1_FOSSIL_FUEL_ETF] price(166.8), volume(10), aggressor_side(ask), buyer(), seller()
[C1_FOSSIL_FUEL_ETF] price(167.5), volume(15), aggressor_side(bid), buyer(), seller()
[C1_FOSSIL_FUEL_ETF] price(166.8), volume(10), aggressor_side(ask), buyer(), seller()
[C1_FOSSIL_FUEL_ETF] price(167.5), volume(10), aggressor_side(bid), buyer(), seller()
[C1_FOSSIL_FUEL_ETF] price(166.8), volume(10), aggressor_side(ask), buyer(), seller()
[C1_FOSSIL_FUEL_ETF] price(167.5), volume(10), aggress

2021-09-25 10:55:45,070 [client    ] [Thread-5    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [25]:
# Returns all public tradeticks since the instantiation of the Exchange
tradeticks = e.get_trade_tick_history(instrument_id)
for t in tradeticks:
    print(t)

In [26]:
# See all your outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    print(f"Outstanding order: order_id({o.order_id}), instrument_id({o.instrument_id}), price({o.price}), volume({o.volume}), side({o.side})")

## Inserting and Deleting Orders

In [27]:
# Insert bid limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to buy.
result = e.insert_order(instrument_id, price=3000, volume=21, side='bid', order_type='limit')
print(f"Order Id: {result}")

Order Id: 1380441


In [28]:
# Insert ask limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=3000, volume=1, side='ask', order_type='limit')
print(f"Order Id: {result}")

Order Id: 1380601


In [29]:
# Insert bid IOC - This can trade against any resting volume but does not remain in the book
# Use this to buy.
result = e.insert_order(instrument_id, price=445.0, volume=1, side='bid', order_type='ioc')
print(f"Order Id: {result}")

Order Id: 1380622


In [30]:
# Insert ask IOC - This can trade against any resting volume but does not remain in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=430.0, volume=1, side='ask', order_type='ioc')
print(f"Order Id: {result}")

Order Id: 1380648


In [31]:
# Attempt to delete inserted order by order_id
order_id = 4
result = e.delete_order(instrument_id, order_id=order_id)
print()
print(f"Deleted order id {order_id}: {result}")


Deleted order id 4: False


In [32]:
# Change volume for existing order
order_id = 5
new_volume = 16
result = e.amend_order(instrument_id, order_id=order_id, volume=new_volume)
print(f"Changed volume for order id {order_id} to {new_volume} lots: {result}")

Changed volume for order id 5 to 16 lots: False


In [33]:
# Delete all outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    result = e.delete_order(instrument_id, order_id=o.order_id)
    print(f"Deleted order id {o.order_id}: {result}")

Deleted order id 1380601: True


## 'Hack' Out of Positions

In [34]:
# Get out of all positions you are currently holding, regarless of the loss involved. That means selling whatever
# you are long, and buying-back whatever you are short. Be sure you know what you are doing when you use this logic.
print(e.get_positions())
for s, p in e.get_positions().items():
    if p > 0:
        e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
    elif p < 0:
        e.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')  
print(e.get_positions())

{'C1_GAS_INC': 0, 'C1_OIL_CORP': 0, 'C1_FOSSIL_FUEL_ETF': 22, 'C2_GREEN_ENERGY_ETF': 0, 'C2_SOLAR_CO': 0, 'C2_WIND_LTD': 0}
{'C1_GAS_INC': 0, 'C1_OIL_CORP': 0, 'C1_FOSSIL_FUEL_ETF': 0, 'C2_GREEN_ENERGY_ETF': 0, 'C2_SOLAR_CO': 0, 'C2_WIND_LTD': 0}
